In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed

In [3]:
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.sga_335 import ( 
        extract_sga_335
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.corte_excel import ( 
        extract_corte_excel
)

In [4]:

from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_335 import ( 
    preprocess_335
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_380 import ( 
    preprocess_380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.cuismp_sharepoint import ( 
    preprocess_df_cid_cuismp_sharepoint
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.corte_excel import ( 
    preprocess_corte_excel
)

In [5]:
# MERGE IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.excel_sga.excel_sga import ( 
merge_sga_335_corte_excel_sharepoint_cuismp_sga380
)

In [6]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_1.o3_fin_inicio_HHMM import validation_fin_inicio_HHMM
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_1.o3_fin_inicio_HHMM import build_failure_messages_diff_fin_inicio_HHMM

In [7]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 1 - 23 04 AL 28 04.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_23-04-2025_29-04-2025_20250528_174144.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_23-04-2025_29-04-2025_20250528_174720.xlsx"

In [8]:
#df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=0, engine="openpyxl")
df_corte_excel = extract_corte_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=0)
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335)
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)

In [9]:
df_corte_excel = preprocess_corte_excel(df_corte_excel)
df_cid_cuismp_sharepoint = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
df_sga_dinamico_335 = preprocess_335(df_sga_dinamico_335)
df_sga_dinamico_380 = preprocess_380(df_sga_dinamico_380)

In [ ]:
row = df_corte_excel[df_corte_excel['nro_incidencia'] == '21810606']
row

In [ ]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )
df_matched_corte_sga335_Sharepoint_cuismp_sga380.shape

In [ ]:
df_vali = validation_fin_inicio_HHMM(df_matched_corte_sga335_Sharepoint_cuismp_sga380)
df_vali.shape 

In [ ]:
row = df_vali[df_vali['nro_incidencia'] == '21810606']
row

In [ ]:
df_mess = build_failure_messages_diff_fin_inicio_HHMM(df_vali)
df_mess